In [1]:
import requests
import urllib3
from bs4 import BeautifulSoup
import re
import json

url = 'https://www.mar.mil.br/hotsites/amazonia_azul/artigos.html'
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

response = requests.get(url, verify=False)
html_content = response.content.decode('utf-8')

soup = BeautifulSoup(html_content, 'html.parser')

links = []
for link in soup.find_all("a", href=re.compile(r"^http://www.repositorio.mar.mil.br")):
    link_url = link["href"]
    https_url = link_url.replace("http://", "https://")
    links.append(https_url)


# Lista para armazenar os JSONs
json_list = []

for link in links:
    response = requests.get(link, verify=False)
    html_content = response.content.decode('utf-8')
    soup = BeautifulSoup(html_content, "html.parser")

    # Lista para armazenar os dicionários de campo e valor
    table_data = {}

    # Encontrar todas as tags de tabela
    tables = soup.find_all("table")

    for table in tables:
        # Encontrar todas as tags <tr> dentro da tabela
        rows = table.find_all("tr")

        
        for row in rows:
            # Encontrar as tags de campo e valor usando as classes metadataFieldLabel e metadataFieldValue
            field = row.find("td", class_="metadataFieldLabel")
            value = row.find("td", class_="metadataFieldValue")

            if field and value:
                table_data[field.text.strip()] = value.text.strip()

                if field.text.strip() == 'URI:':
                    uri_list = re.findall(r"(http?://\S+?)(?=http?://|\s|$)", value.text.strip())
                    links_https = []
                    for link in uri_list:
                        links_https.append(link.replace("http://", "https://"))
                    table_data[field.text.strip()] = links_https

    pdf_links = {f"https://www.repositorio.mar.mil.br{a['href']}" for a in soup.find_all("a", href=re.compile(r"/bitstream.*\.pdf$"))}
    table_data['PDFs'] = list(pdf_links)

    # Adicionar a lista de campo e valor como um JSON à lista de JSONs
    json_list.append(table_data)

# Imprimir cada JSON da lista em uma linha separada e com indentação
for json_data in json_list:
    print(json.dumps(json_data, ensure_ascii=False, indent=4))

{
    "Título:": "A Amazônia Azul e a geopolítica brasileira: a Marinha do Brasil na defesa da Amazônia Azul",
    "Autor(es):": "Medeiros, Jorge Luiz Mesquita de",
    "Orientador(es):": "Medeiros, Sabrina Evangelista",
    "Palavras-chave:": "Assuntos Militares em GeralCiência MilitarAmazônia Azul",
    "Data do documento:": "2008",
    "Editor:": "Escola de Guerra Naval (EGN)",
    "Descrição:": "EGN:Monografia apresentada à Escola de Guerra Naval, como requisito parcial para conclusão do Curso de Estado-Maior para Oficiais Superiores - C-EMOS.",
    "Tipo de Acesso:": "Acesso aberto",
    "URI:": [
        "https://www.redebim.dphdm.mar.mil.br/vinculos/00000f/00000fd6.pdf",
        "https://www.repositorio.mar.mil.br/handle/ripcmb/29901"
    ],
    "Tipo:": "Trabalho de fim de curso",
    "Aparece nas coleções:": "Defesa Nacional: Coleção de Trabalhos de Conclusão de CursoEstudos Militares: Coleção de Trabalhos de Conclusão de Curso",
    "PDFs": [
        "https://www.repositorio.